# This is all over the place, don't worry will be fixed

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
import string
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,\
HashingVectorizer
%matplotlib inline

Loading in the data

In [2]:
df= pd.read_csv('CraigslistCars.csv')

In [3]:
df_1 = df.copy()

In [ ]:
df_1.head()
# visually seeing the first 5 rows of the raw dataset

From the first few lines we are seeing that there are many rows that have NaN as a value within a column.

In [ ]:
# Will take a random sample of a row to see if an arbritrary row will also follow suit of the first 5 rows by having 
# multiple NaN values
df_1.sample()

As you may possibly see above at least one column could have a NaN value in a row.

In [ ]:
# Want to see what are the 25 columns that will be dealt with. Also will see which should be the target for this dataset
df_1.columns

Seeing that within the dataset that price exists and therefore should be a solid target to predict.

In [ ]:
# Seeing the range of the prices
df_1.price.min(), df_1.price.max()

In [ ]:
# The range of price within the dataset goes from 0 to 3,736,928,711
# Want to check out the years of these vehicles because some older cars have been valued in the millions
df_1.loc[df_1['price'] == df_1.price.min(), ['year']], df_1.loc[df_1['price']==df_1.price.max(), ['year']]

Seeing that over 30,000 rows have the price of 0 and 2 have the large value it is showing the dataset is far fromm perfect

Best way to have the price of 0 not sway modeling later on is to limit price values as well as these strange values that are in the billions. Therefore will take an inner 90% of the data.

In [ ]:
df_1['price'] = df_1['price'][df_1['price'].between(df_1['price'].quantile(.085), df_1['price'].quantile(.985),inclusive=True)]
df_1.price.min(), df_1.price.max()
# shows 100 - 59999 is an inner 90%%

In [ ]:
# Want new df with just price values that range from 100 to 59999

In [4]:
# Making new df of just values of cars between 100 and 59,999
df_2 = df_1.loc[(df_1['price']>100) & (df_1['price'] <59999)].copy()
df_2.head(), len(df_2)

(           id                                                url  \
 0  7222695916  https://prescott.craigslist.org/cto/d/prescott...   
 1  7218891961  https://fayar.craigslist.org/ctd/d/bentonville...   
 2  7221797935  https://keys.craigslist.org/cto/d/summerland-k...   
 3  7222270760  https://worcester.craigslist.org/cto/d/west-br...   
 4  7210384030  https://greensboro.craigslist.org/cto/d/trinit...   
 
                    region                         region_url  price  year  \
 0                prescott    https://prescott.craigslist.org   6000   NaN   
 1            fayetteville       https://fayar.craigslist.org  11900   NaN   
 2            florida keys        https://keys.craigslist.org  21000   NaN   
 3  worcester / central MA   https://worcester.craigslist.org   1500   NaN   
 4              greensboro  https://greensboro.craigslist.org   4900   NaN   
 
   manufacturer model condition cylinders  ... size  type paint_color  \
 0          NaN   NaN       NaN       NaN

# Dealing with all the possible NaN values in the dataset

In [ ]:
df_2.isna().sum()

year, manufacturer, model, condition, cylinders, fuel, odometer, title_status, transmission, VIN, drive, size, type, paint_color
image_url, description, county, state, lat, long, posting_date have Na values and some have around 40% of its rows with NaN values.

In [5]:
# Checking the 'county' column since it has the most NaN values.
df_2['county'].value_counts()
# is completely empty

Series([], Name: county, dtype: int64)

In [5]:
# The column is not going to be useful therefore it is safe to just drop it
df_2.drop('county', axis=1, inplace=True)
df_2.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


County dealt with

The 'year' column is the first column that shows it has NaN values so lets take a deeper dive into the column

In [ ]:
df_2['year'].value_counts()

In [ ]:
# Checking the oldest car
df_2['year'].min()

In [ ]:
# Checking newest car
df_2.year.max()

In [ ]:
# Making a sorted list of all the years of the cars represented in the dataset
unique_years = df_2['year'].unique()
print(sorted(unique_years))

Barring a couple of years in the early 1900's, the dataset seems to have nearly every year in the 20th century and all years that have occured in the 21st century.

In [ ]:
df_2.year.value_counts(normalize=True)

In [ ]:
df_top_years = df_2.copy()
df_top_years['year'] = df_top_years['year'][df_top_years['year'].between(df_top_years['year'].quantile(.1), df_top_years['year'].quantile(1),inclusive=True)]
df_top_years.year.min(), df_top_years.year.max()

In [6]:
df_limited_years= df_2.loc[df_2['year'] >2002].copy()

In [ ]:
df_limited_years.year.value_counts()

In [ ]:
df_limited_years.isna().sum()

Other columns

In [7]:
df_condition = df_limited_years.loc[df_limited_years['condition'].notna()].copy()
df_model_and_condition = df_condition.loc[df_condition['model'].notna()].copy()
condition_dict = {}
for a, b in df_model_and_condition[['model', 'condition']].values:
    condition_dict[a] = b

In [8]:
df_limited_years['condition'] = df_limited_years.apply(lambda x: x['condition'] if pd.isna(x['model']) or pd.notna(x['condition']) else condition_dict[x['model']] if x['model'] in condition_dict.keys() else x['condition'], axis=1)

In [9]:
df_limited_years.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer     10234
model             2530
condition        17764
cylinders       144503
fuel              2226
odometer          2050
title_status      7004
transmission      1645
VIN             120074
drive           105630
size            253876
type             69704
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [10]:
df_cylinders = df_limited_years.loc[df_limited_years['cylinders'].notna()].copy()
df_cylinders_model = df_cylinders.loc[df_cylinders['model'].notna()].copy()
cylinders_dictionary = {}
for c, d in df_cylinders_model[['model', 'cylinders']].values:
    cylinders_dictionary[c] = d

In [11]:
df_limited_years['cylinders'] = df_limited_years.apply( lambda x: x['cylinders'] if pd.isna( x['model'] ) or pd.notna( x['cylinders'] ) else cylinders_dictionary[x['model']] if x['model'] in cylinders_dictionary.keys() else x['cylinders'], axis=1)

In [12]:
df_manufacturer = df_limited_years.loc[df_limited_years['manufacturer'].notna()].copy()
df_manufacturer_model = df_manufacturer.loc[df_manufacturer['model'].notna()].copy()
manufacturer_dictionary = {}
for e, f in df_manufacturer_model[['model', 'manufacturer']].values:
    manufacturer_dictionary[e] = f

In [13]:
df_limited_years['manufacturer'] = df_limited_years.apply( lambda x: x['manufacturer'] if pd.isna( x['model'] ) or pd.notna( x['manufacturer'] ) else manufacturer_dictionary[x['model']] if x['model'] in manufacturer_dictionary.keys() else x['manufacturer'], axis=1)

In [14]:
df_limited_years.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer      9789
model             2530
condition        17764
cylinders        34854
fuel              2226
odometer          2050
title_status      7004
transmission      1645
VIN             120074
drive           105630
size            253876
type             69704
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [15]:
df_limited_years['cylinders'] = df_limited_years.groupby('model')['cylinders'].ffill()
df_limited_years['manufacturer'] = df_limited_years.groupby('model')['manufacturer'].ffill()
df_limited_years['fuel']=df_limited_years.groupby('model')['fuel'].ffill()
df_limited_years['odometer'] = df_limited_years['odometer'].fillna(df_limited_years.groupby(['year','model'])['odometer'].transform('mean'))
df_limited_years['size']= df_limited_years.groupby('model')['size'].ffill()
df_limited_years['type']= df_limited_years.groupby('model')['type'].ffill()
df_limited_years['manufacturer']= df_limited_years.groupby('model')['manufacturer'].ffill()

In [ ]:
df_limited_years.isna().sum()

In [16]:
unique_manufacturer = df_2['manufacturer'].unique()
manufacturer_list = list(unique_manufacturer)
manufacturer_list =manufacturer_list[1:]
type(manufacturer_list)

list

In [ ]:
manufacturer_list

Maybe keep these to show how the description had the manufacturer's name

In [ ]:
# need to find a way to fill in missing manufacturers. too many Na's to drop

In [ ]:
# see if any other columns have the manufacturer name 
# list is in and return true or false

In [ ]:
# sample = df_2['description'][97]

In [ ]:
# len(sample)

In [ ]:
# pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
# tokenizer = RegexpTokenizer(pattern)
# sample_doc = tokenizer.tokenize(sample)
# sample_doc = [token.lower() for token in sample_doc]
# sw =stopwords.words('english')
# sample_doc = [token for token in sample_doc if token not in sw]

In [ ]:
# sample_doc

In [ ]:
# len(set(sample_doc))

In [ ]:
# print(set(sample_doc))

In [ ]:
# type(sample_doc)

In [ ]:
# # List1
# List1 = unique_manufacturer
 
# # List2
# List2 = sample_doc

# check =  any(item in List1 for item in List2)
 
# if check is True:
#     print("The list {} contains all elements of the list {}".format(List1, List2))    
# else :
#     print("No, List1 doesn't have all elements of the List2.")

In [ ]:
# def list_contains(List1, List2): 
#     check = False
  
#     # Iterate in the 1st list 
#     for m in List1: 
  
#         # Iterate in the 2nd list 
#         for n in List2: 
    
#             # if there is a match
#             if m == n: 
#                 check = True
#                 return check  
                  
#     return check 
      
# # Test Case 1
# List1 = unique_manufacturer
# List2 = sample_doc 
# print("Test Case#1 ", list_contains(List1, List2)) 

In [ ]:
# car = set(unique_manufacturer) & set(sample_doc)
# car

In [ ]:
# df_2['description']=df_2['description'].str.lower()

In [ ]:
# df_2.sample(10)

In [ ]:
# df_2.sample()

In [ ]:
# df_2['exists'] = df.drop('manufacturer', 1).isin(df['manufacturer']).any(1) 
# df_2.exists.value_counts()

In [ ]:
# Only 8 cases where the name of the manufacturer's name is mentioned outside of the manufacturers column therefore wan't
# able to fill in the Na Values

In [17]:
manu_desc=df_limited_years.loc[df_limited_years['manufacturer'].isna(), ['description','manufacturer', 'id']].copy()
manu_desc.head()

,description,manufacturer,id
97,Carvana is the safer way to buy a car During t...,NaN,7309271279
122,"2005 Blue Bird 12 window, 72 passenger bus tha...",NaN,7306198330
135,Carvana is the safer way to buy a car During t...,NaN,7305150004
155,Carvana is the safer way to buy a car During t...,NaN,7302642422
220,2006 STERLING ACTERRA GRAPPLE TRUCK PAC-MAC LO...,NaN,7316423933


In [18]:
manu_desc['description']=manu_desc['description'].str.lower().str.split()

In [19]:
manu_desc.head()

,description,manufacturer,id
97,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7309271279
122,"[2005, blue, bird, 12, window,, 72, passenger,...",NaN,7306198330
135,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7305150004
155,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7302642422
220,"[2006, sterling, acterra, grapple, truck, pac-...",NaN,7316423933


In [20]:
index_nums=[]
m =[]
for index, row in manu_desc.iterrows():
    for manufacturer in manufacturer_list:
        if manufacturer in row['description']:
            index_nums.append(index)
            m.append(manufacturer)            

In [21]:
# result = [None]*(len(index_nums)+len(m))
# result[::2] = index_nums
# result[1::2] =m

# for x, y in zip(result[::2], result[1::2]):
    
#     print(x,y)


In [22]:
manu_desc_completed = pd.DataFrame(index_nums, columns=['index'])
manu_desc_completed['manufacturer'] = m

In [23]:
manu_desc_completed.head(15)

,index,manufacturer
0,220,gmc
1,220,ford
2,601,gmc
3,696,gmc
4,696,chevrolet
5,696,ford
6,696,nissan
7,696,dodge
8,696,mini
9,787,gmc


In [24]:
manu_desc_completed_new=manu_desc_completed.drop_duplicates(subset=['index'])
manu_desc_completed_new.manufacturer.value_counts()

gmc              869
ford             297
toyota           228
chevrolet        185
dodge            105
volvo             95
hyundai           84
honda             83
nissan            78
chrysler          65
jeep              62
subaru            49
ram               44
bmw               42
infiniti          42
mercedes-benz     41
volkswagen        41
mini              39
mazda             33
mitsubishi        30
cadillac          27
morgan            26
lexus             24
ferrari           19
audi              16
lincoln           16
pontiac           15
acura             15
kia               13
mercury            5
rover              4
jaguar             4
buick              4
tesla              4
porsche            4
fiat               4
saturn             4
Name: manufacturer, dtype: int64

In [25]:
manu_desc_completed_new.set_index('index',inplace=True )
manu_desc_completed_new.head()

,manufacturer
index,
220,gmc
601,gmc
696,gmc
787,gmc
794,gmc


In [26]:
df_yo= df_limited_years.copy()

In [27]:
df_yo.update(manu_desc_completed_new)
df_yo.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer      7073
model             2530
condition        17764
cylinders        33444
fuel              1410
odometer           392
title_status      7004
transmission      1645
VIN             120074
drive           105630
size             91605
type              9966
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [ ]:
df_yo.iloc[170:180]

In [28]:
unique_condition = df_2['condition'].unique()
condition_list = list(unique_condition)
condition_list =condition_list[1:]
condition_list

['good', 'excellent', 'fair', 'like new', 'new', 'salvage']

In [29]:
cond_desc=df_limited_years.loc[df_limited_years['condition'].isna(), ['description','condition', 'id']].copy()
cond_desc.head()

,description,condition,id
195,2012 Black 140-inch Dodge Charger Limousine #1...,NaN,7316711193
225,2011 Chevrolet Silverado 1500 LTZ Crew Cab 4WD...,NaN,7316396026
226,2015 Chevrolet Silverado 1500 LT Crew Cab 4WD ...,NaN,7316395026
227,2014 Chevrolet Silverado 1500 2LT Crew Cab Lon...,NaN,7316393935
265,"2006 *Suzuki* *XL-7* SUV - $2,975Call Us Toda...",NaN,7315970659


In [27]:
cond_desc['description']=cond_desc['description'].str.lower().str.split()

In [30]:
index_nums2=[]
c =[]
for index, row in cond_desc.iterrows():
    for condition in condition_list:
        if condition in row['description']:
            index_nums2.append(index)
            c.append(condition)  

In [ ]:
# result2 = [None]*(len(index_nums2)+len(c))
# result2[::2] = index_nums2
# result2[1::2] =c

# for x, y in zip(result2[::2], result2[1::2]):
    
#     print(x,y)

In [31]:
cond_desc_completed = pd.DataFrame(index_nums2, columns=['index'])
cond_desc_completed['condition'] = c
cond_desc_completed_new=cond_desc_completed.drop_duplicates(subset=['index'])
cond_desc_completed_new.set_index('index',inplace=True )
cond_desc_completed_new.head()

,condition
index,
414,like new
460,new
904,good
905,good
907,good


In [32]:
df_yo.update(cond_desc_completed_new)
df_yo.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer      7073
model             2530
condition        12162
cylinders        33444
fuel              1410
odometer           392
title_status      7004
transmission      1645
VIN             120074
drive           105630
size             91605
type              9966
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [ ]:
# unique_cylinders = df_2['cylinders'].unique()
# cylinders_list = list(unique_cylinders)
# cylinders_list = cylinders_list[1:]
# cylinders_list

In [ ]:
# cyld_desc=df_limited_years.loc[df_limited_years['cylinders'].isna(), ['description','cylinders', 'id']].copy()
# cyld_desc.head()

In [ ]:
# cyld_desc['description']=cyld_desc['description'].str.lower().str.split()

In [ ]:

# ll=[]
# cy =[]
# for index, row in cyld_desc.iterrows():
#     for cylinders in cylinders_list:
#         if cylinders in row['description']:
#             ll.append(index)
#             cy.append(manufacturer)  
# result3 = [None]*(len(ll)+len(cy))
# result3[::2] = ll
# result3[1::2] =cy
# result3
# # for x, y in zip(result3[::2], result3[1::2]):
    
# #     print(x,y)

In [33]:
unique_fuel = df_limited_years['fuel'].unique()
fuel_list = list(unique_fuel)
fuel_list = ['gas', 'diesel', 'hybrid', 'electric']

In [34]:
fuel_desc=df_limited_years.loc[df_limited_years['fuel'].isna(), ['description','fuel', 'id']].copy()
fuel_desc['description']=fuel_desc['description'].str.lower().str.split()
fuel_desc.head()

,description,fuel,id
225,"[2011, chevrolet, silverado, 1500, ltz, crew, ...",NaN,7316396026
226,"[2015, chevrolet, silverado, 1500, lt, crew, c...",NaN,7316395026
227,"[2014, chevrolet, silverado, 1500, 2lt, crew, ...",NaN,7316393935
314,"[2011, *jeep*, *grand, cherokee*, rwd, 4dr, la...",NaN,7315518138
368,"[2010, *chevrolet*, *hhr*, fwd, 4dr, lt, w/1lt...",NaN,7314945978


In [35]:
index_nums4=[]
f =[]
for index, row in fuel_desc.iterrows():
    for fuel in fuel_list:
        if fuel in row['description']:
            index_nums4.append(index)
            f.append(fuel)  

In [ ]:
# result4 = [None]*(len(index_nums4)+len(f))
# result4[::2] = index_nums4
# result4[1::2] =f

# for x, y in zip(result4[::2], result4[1::2]):
    
#     print(x,y)

In [36]:
fuel_desc_completed = pd.DataFrame(index_nums4, columns=['index'])
fuel_desc_completed['fuel'] = f
fuel_desc_completed_new=fuel_desc_completed.drop_duplicates(subset=['index'])
fuel_desc_completed_new.set_index('index',inplace=True )
fuel_desc_completed_new.head()

,fuel
index,
1569,hybrid
2278,hybrid
3621,hybrid
4952,hybrid
10399,electric


In [37]:
df_yo.update(fuel_desc_completed_new)
df_yo.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer      7073
model             2530
condition        12162
cylinders        33444
fuel              1215
odometer           392
title_status      7004
transmission      1645
VIN             120074
drive           105630
size             91605
type              9966
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [38]:
unique_transmission = df_limited_years['transmission'].unique()
trasnmission_list = list(unique_transmission)
transmission_list = ['automatic', 'manual']

In [39]:
tran_desc=df_limited_years.loc[df_limited_years['transmission'].isna(), ['description','transmission', 'id']].copy()
tran_desc['description']=tran_desc['description'].str.lower().str.split()
tran_desc.head()

,description,transmission,id
2961,"[2016, nissan, murano, sv, offered, by:, autov...",NaN,7312867213
5088,"[2012, ford, f-550, super, duty, 4x4, 2dr, reg...",NaN,7316461977
5250,"[bad, credit, -, no, credit..., we, can, help!...",NaN,7315616467
5284,"[2006, ford, f-550, super, duty, 4x4, 4dr, sup...",NaN,7315552489
5384,"[this, chevrolet, corvette, can, be, yours, to...",NaN,7315160850


In [40]:
index_nums5=[]
t =[]
for index, row in tran_desc.iterrows():
    for transmission in transmission_list:
        if transmission in row['description']:
            index_nums5.append(index)
            t.append(transmission)  

In [ ]:
# result5 = [None]*(len(index_nums5)+len(t))
# result5[::2] = index_nums5
# result5[1::2] =t

# for x, y in zip(result5[::2], result5[1::2]):
    
#     print(x,y)

In [41]:
tran_desc_completed = pd.DataFrame(index_nums5, columns=['index'])
tran_desc_completed['transmission'] = t
tran_desc_completed_new=tran_desc_completed.drop_duplicates(subset=['index'])
tran_desc_completed_new.set_index('index',inplace=True )
tran_desc_completed_new.head()

,transmission
index,
8793,automatic
8800,automatic
8807,automatic
8808,automatic
8810,automatic


In [42]:
df_yo.update(tran_desc_completed_new)
df_yo.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer      7073
model             2530
condition        12162
cylinders        33444
fuel              1215
odometer           392
title_status      7004
transmission      1158
VIN             120074
drive           105630
size             91605
type              9966
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [43]:
unique_drive = df_limited_years['drive'].unique()
drive_list = list(unique_drive)
drive_list= drive_list[1:]
drive_list

['rwd', '4wd', 'fwd']

In [44]:
driv_desc=df_limited_years.loc[df_limited_years['drive'].isna(), ['description','drive', 'id']].copy()
driv_desc['description']=driv_desc['description'].str.lower().str.split()
driv_desc.head()

,description,drive,id
27,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316814884
28,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316814758
29,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316814989
30,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316743432
39,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7315770394


In [45]:
driv_desc.isna().sum()

description         1
drive          105630
id                  0
dtype: int64

In [46]:
driv_desc.dropna(subset=['description'], axis=0, inplace=True)

In [47]:
index_nums6=[]
d =[]
for index, row in driv_desc.iterrows():
    for drive in drive_list:
        if drive in row['description']:
            index_nums6.append(index)
            d.append(drive)  

In [ ]:
# result6 = [None]*(len(index_nums6)+len(d))
# result6[::2] = index_nums6
# result6[1::2] =d

# for x, y in zip(result6[::2], result6[1::2]):
    
#     print(x,y)

In [48]:
driv_desc_completed = pd.DataFrame(index_nums6, columns=['index'])
driv_desc_completed['drive'] = d
driv_desc_completed_new=driv_desc_completed.drop_duplicates(subset=['index'])
driv_desc_completed_new.set_index('index',inplace=True )
driv_desc_completed_new.head()

,drive
index,
423,4wd
504,4wd
510,4wd
659,4wd
1038,4wd


In [49]:
df_yo.update(driv_desc_completed_new)
df_yo.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer      7073
model             2530
condition        12162
cylinders        33444
fuel              1215
odometer           392
title_status      7004
transmission      1158
VIN             120074
drive            93792
size             91605
type              9966
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [50]:
unique_size = df_limited_years['size'].unique()
size_list = list(unique_size)
size_list= size_list[1:]
size_list

['full-size', 'mid-size', 'compact', 'sub-compact']

In [51]:
size_desc=df_limited_years.loc[df_limited_years['size'].isna(), ['description','size', 'id']].copy()
size_desc['description']=size_desc['description'].str.lower().str.split()
size_desc.head()

,description,size,id
27,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316814884
28,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316814758
29,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316814989
30,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316743432
32,"[carvana, is, the, safer, way, to, buy, a, car...",NaN,7316343444


In [52]:
index_nums7=[]
s =[]
for index, row in size_desc.iterrows():
    for size in size_list:
        if size in row['description']:
            index_nums7.append(index)
            s.append(size)  

In [ ]:
# result7 = [None]*(len(index_nums7)+len(s))
# result7[::2] = index_nums7
# result7[1::2] =s

# for x, y in zip(result7[::2], result7[1::2]):
    
#     print(x,y)

In [53]:
size_desc_completed = pd.DataFrame(index_nums7, columns=['index'])
size_desc_completed['size'] = s
size_desc_completed_new=size_desc_completed.drop_duplicates(subset=['index'])
size_desc_completed_new.set_index('index',inplace=True )
size_desc_completed_new.head()

,size
index,
248,compact
294,compact
583,compact
727,compact
874,full-size


In [54]:
df_yo.update(size_desc_completed_new)
df_yo.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer      7073
model             2530
condition        12162
cylinders        33444
fuel              1215
odometer           392
title_status      7004
transmission      1158
VIN             120074
drive            93792
size             88753
type              9966
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [ ]:
df_yo['type'].value_counts()

In [55]:
unique_type = df_limited_years['type'].unique()
type_list = list(unique_type)
# type_list= type_list[1:]
type_list = ['pickup',
 'truck',
 'other',
 'coupe',
 'SUV',
 'hatchback',
 'mini-van',
 'sedan',
 'offroad',
 'bus',
 'convertible',
 'wagon',
 'van']

In [56]:
type_desc=df_limited_years.loc[df_limited_years['type'].isna(), ['description','type', 'id']].copy()
type_desc['description']=type_desc['description'].str.lower().str.split()
type_desc.head()

,description,type,id
195,"[2012, black, 140-inch, dodge, charger, limous...",NaN,7316711193
196,"[2005, ford, focus, 189k, miles, automatic, tr...",NaN,7316707537
200,"[phone:, ☎, (205), 862-8003, text:, text, (205...",NaN,7316694884
201,"[phone:, ☎, (205), 862-8003, text:, text, (205...",NaN,7316694885
203,"[contemporary, automotive, address:, 3427, sky...",NaN,7316691449


In [57]:
index_nums8=[]
ty =[]
for index, row in type_desc.iterrows():
    for types in type_list:
        if types in row['description']:
            index_nums8.append(index)
            ty.append(types)  

In [ ]:
# result8 = [None]*(len(index_nums8)+len(ty))
# result8[::2] = index_nums8
# result8[1::2] =ty

# for x, y in zip(result8[::2], result8[1::2]):
    
#     print(x,y)

In [58]:
type_desc_completed = pd.DataFrame(index_nums8, columns=['index'])
type_desc_completed['type'] = ty
type_desc_completed_new=type_desc_completed.drop_duplicates(subset=['index'])
type_desc_completed_new.set_index('index',inplace=True )
type_desc_completed_new.head()

,type
index,
201,truck
204,pickup
206,truck
220,truck
229,truck


In [59]:
df_yo.update(type_desc_completed_new)
df_yo.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer      7073
model             2530
condition        12162
cylinders        33444
fuel              1215
odometer           392
title_status      7004
transmission      1158
VIN             120074
drive            93792
size             88753
type              4586
paint_color     101144
image_url            0
description          2
state                0
lat               3283
long              3283
posting_date         0
dtype: int64

In [60]:
df_yo.drop(['id', 'url', 'region_url', 'VIN', 'image_url', 'description', 'lat', 'long', 'posting_date'], axis=1, inplace=True)
df_yo.head()

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
27,auburn,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8 cylinders,gas,57923.0,clean,other,NaN,NaN,pickup,white,al
28,auburn,22590,2010.0,chevrolet,silverado 1500,good,8 cylinders,gas,71229.0,clean,other,NaN,NaN,pickup,blue,al
29,auburn,39590,2020.0,chevrolet,silverado 1500 crew,good,8 cylinders,gas,19160.0,clean,other,NaN,NaN,pickup,red,al
30,auburn,30990,2017.0,toyota,tundra double cab sr,good,8 cylinders,gas,41124.0,clean,other,NaN,NaN,pickup,red,al
31,auburn,15000,2013.0,ford,f-150 xlt,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,full-size,truck,black,al


In [61]:
df_yo1 = df_yo.copy()
df_yo1.dropna(inplace=True)
df_yo1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151995 entries, 31 to 426872
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        151995 non-null  object 
 1   price         151995 non-null  int64  
 2   year          151995 non-null  float64
 3   manufacturer  151995 non-null  object 
 4   model         151995 non-null  object 
 5   condition     151995 non-null  object 
 6   cylinders     151995 non-null  object 
 7   fuel          151995 non-null  object 
 8   odometer      151995 non-null  float64
 9   title_status  151995 non-null  object 
 10  transmission  151995 non-null  object 
 11  drive         151995 non-null  object 
 12  size          151995 non-null  object 
 13  type          151995 non-null  object 
 14  paint_color   151995 non-null  object 
 15  state         151995 non-null  object 
dtypes: float64(2), int64(1), object(13)
memory usage: 19.7+ MB


In [64]:
# import pickle
# with open('cars.pkl', 'wb') as carpickle:
#     pickle.dump(df_yo1,carpickle)

In [62]:
df_yo2=df_yo.copy()

In [63]:
df_yo2.drop(['drive', 'size', 'paint_color'], axis=1,inplace=True)
df_yo2.dropna(inplace=True)
df_yo2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 291525 entries, 27 to 426878
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        291525 non-null  object 
 1   price         291525 non-null  int64  
 2   year          291525 non-null  float64
 3   manufacturer  291525 non-null  object 
 4   model         291525 non-null  object 
 5   condition     291525 non-null  object 
 6   cylinders     291525 non-null  object 
 7   fuel          291525 non-null  object 
 8   odometer      291525 non-null  float64
 9   title_status  291525 non-null  object 
 10  transmission  291525 non-null  object 
 11  type          291525 non-null  object 
 12  state         291525 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 31.1+ MB


In [64]:
df_yo2.head()

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,type,state
27,auburn,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8 cylinders,gas,57923.0,clean,other,pickup,al
28,auburn,22590,2010.0,chevrolet,silverado 1500,good,8 cylinders,gas,71229.0,clean,other,pickup,al
29,auburn,39590,2020.0,chevrolet,silverado 1500 crew,good,8 cylinders,gas,19160.0,clean,other,pickup,al
30,auburn,30990,2017.0,toyota,tundra double cab sr,good,8 cylinders,gas,41124.0,clean,other,pickup,al
31,auburn,15000,2013.0,ford,f-150 xlt,excellent,6 cylinders,gas,128000.0,clean,automatic,truck,al


In [61]:
# with open('cars_witthout_drive_size_paint_NaN.pkl', 'wb') as carpickles:
#     pickle.dump(df_yo2,carpickles)

NameError: name 'pickle' is not defined

In [65]:
df_yo1['paint_color'].value_counts()

white     37949
black     30883
silver    23550
grey      16847
blue      15587
red       14557
custom     4249
brown      3388
green      2962
orange      847
yellow      839
purple      365
Name: paint_color, dtype: int64

In [62]:
df_yo1.head()

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
31,auburn,15000,2013.0,ford,f-150 xlt,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,full-size,truck,black,al
55,auburn,19900,2004.0,ford,f250 super duty,good,8 cylinders,diesel,88000.0,clean,automatic,4wd,full-size,pickup,blue,al
59,auburn,14000,2012.0,honda,odyssey,excellent,6 cylinders,gas,95000.0,clean,automatic,fwd,full-size,mini-van,silver,al
73,auburn,15000,2017.0,dodge,charger rt 4dr sedan,excellent,8 cylinders,gas,90000.0,rebuilt,automatic,rwd,mid-size,sedan,grey,al
86,auburn,3000,2004.0,chrysler,town & country,good,6 cylinders,gas,176144.0,clean,automatic,fwd,mid-size,mini-van,silver,al


In [ ]:
# sample2 = manu_desc['description']

In [ ]:
# pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
# tokenizer = RegexpTokenizer(pattern)
# sample_doc = tokenizer.tokenize(sample)
# sample_doc = [token.lower() for token in sample_doc]
# sw =stopwords.words('english')
# sample_doc = [token for token in sample_doc if token not in sw]

In [ ]:
# pattern_2 = "([a-zA-Z]+(?:'[a-z]+)?)"
# tokenizer = RegexpTokenizer(pattern_2)

In [ ]:
# pattern_2 = "([a-zA-Z]+(?:'[a-z]+)?)"
# tokenizer = RegexpTokenizer(pattern_2)
# manu_desc['tokenized_sents'] = manu_desc.apply(lambda row: tokenizer.tokenize(row['description']), axis=1)


In [ ]:
# manu_desc['new'] = manu_desc['tokenized_sents'].str.split()
# manu_desc.head()

In [ ]:
# manu_desc['description'] =manu_desc['description'].str.split()
# manu_desc.head()

In [ ]:
# manu_desc['description'] = manu_desc['description'].lower()
# manu_desc.head()

In [ ]:
# manu_desc['description'] = manu_desc['description'].str.lower()

In [ ]:
# manu_desc['exists'] = manu_desc.drop('manufacturer', 1).isin(manu_desc['manufacturer']).any(1) 
# manu_desc.exists.value_counts()

In [ ]:
# manu_desc["test"] = np.where(manu_desc["tokenized_sents"].isin(manufacturer), 'Other')


In [ ]:
# manu_desc['tokenized_sents'].lower()

In [ ]:
# car1 = set(unique_manufacturer) & set(manu_desc['tokenized_sents'][0])
# car1

In [ ]:
# set(manu_desc['tokenized_sents'][0])

In [ ]:
# df_new = manu_desc[manu_desc['tokenized_sents'].isin(['toyota'])]
# df_new.head()